<a href="https://colab.research.google.com/github/jdeiros/soa-2020/blob/master/HPC/Deiros_Jeronimo_ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Ejemplo de OpenMP (Hola Mundo)

In [2]:
# Codigo Python, que tiene el código C en una cadena, que es guardada en el archivo code.cpp.
code = """
// Hola Mundo desde OpenMP, usando c, ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <omp.h>    // Cabecera OpenMP   
  
int main(int argc, char* argv[]) 
{
    std::cout<<"Inicio..."<<std::endl;

    // Region paralela
    #pragma omp parallel
    { 
        std::cout<<"Hola mundo!!!... soy el hilo " << omp_get_thread_num()<< ", de "<<  omp_get_num_threads() <<", procesadores "<< omp_get_num_procs()<< std::endl;  
    }
    // Region paralela

    std::cout<<"Fin..."<<std::endl;
}
"""
text_file = open("code.cpp", "w")
text_file.write(code)
text_file.close()

#1.1.Compilación de código C Hola Mundo.

In [3]:
!g++ -o hello -fopenmp code.cpp


#1.2.Ejecución Hola Mundo en OpenMP

In [5]:
%env OMP_NUM_THREADS=7
!./hello

env: OMP_NUM_THREADS=7
Inicio...
Hola mundo!!!... soy el hilo 2, de 7, procesadores 2
Hola mundo!!!... soy el hilo 0, de 7, procesadores 2
Hola mundo!!!... soy el hilo 6, de 7, procesadores 2
Hola mundo!!!... soy el hilo 5, de 7, procesadores 2
Hola mundo!!!... soy el hilo 4, de 7, procesadores 2
Hola mundo!!!... soy el hilo Hola mundo!!!... soy el hilo 3, de 7, procesadores 2
1, de 7, procesadores 2
Fin...


In [6]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.000
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4400.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

OTRO

In [60]:
# Codigo Python, que tiene el código C de la ejecución Axpy.
code = """
#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP   

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_AXPY_SEC      2
#define HTH_AXPY_OMP      3

// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
  int i,c;
  TIEMPO_INI( HTH_TOTAL )

  // Leo los parametros.
  if( argc != 4 )
  {
      std::cerr<< " Error en los parametros de indicar: (alfa), (Tamanio vector), (ciclos ejecucion)."<<argc<<std::endl;
      exit( -1 );
  }

  float alfa     = atof( argv[1] );
  int cantidad_N = atoi( argv[2] );
  int ciclos     = atoi( argv[3] );
  double suma1 = 0;
  double suma2 = 0;
  // --------------------------------------------
  // Defino la memoria de los vectores.

  std::vector<int> Y( cantidad_N );
  std::vector<int> frecuencias( 10 );
  std::vector<int> frecuencias2( 10 );

  for(int i=0; i<frecuencias.size(); i++)
    frecuencias[i]=0;
  for(int i=0; i<frecuencias2.size(); i++)
    frecuencias2[i]=0;

  srand ( time(NULL) );
  for (int i=0;i<Y.size();i++)
  {
    Y[i] = (int)(rand()%10);
    ++frecuencias[Y[i]];
  }

  // --------------------------------------------
  // Realizo la función Axpy en forma secuencial.

  TIEMPO_INI( HTH_AXPY_SEC )

  
  for(i=0;i<Y.size();i++)
  {
    suma1 += Y[i];
  }
  

  TIEMPO_FIN( HTH_AXPY_SEC )

  // --------------------------------------------
  // Realizo la función Axpy con OpenMP.

  TIEMPO_INI( HTH_AXPY_OMP )

  
  #pragma omp parallel for
  for(i=0;i<Y.size();i++)
  {
    suma2 += Y[i];
    ++frecuencias2[Y[i]];
  }
  

  TIEMPO_FIN( HTH_AXPY_OMP )

  // --------------------------------------------
  // Muestro los resultados.

  std::cout<<" Y : ["; 
  for(i=0;i<Y.size();i++)
  {
    std::cout<<Y[i]<< ", ";
  }
  std::cout<<"]"<<std::endl; 

  std::cout<<" frecuencias : [";
  for(i=0;i<frecuencias.size();i++)
  {
    std::cout<<frecuencias[i]<< ", ";
  }
  std::cout<<"]"<<std::endl; 

  std::cout<<" frecuencias2 : [";
  for(i=0;i<frecuencias2.size();i++)
  {
    std::cout<<frecuencias2[i]<< ", ";
  }
  std::cout<<"]"<<std::endl; 

  std::cout<<"suma1: "<<suma1<<std::endl;
  std::cout<<"media1: "<<suma1/cantidad_N<<std::endl;
  std::cout<<"suma2: "<<suma2<<std::endl; 
  std::cout<<"media2: "<<suma2/cantidad_N<<std::endl;

  TIEMPO_FIN( HTH_TOTAL )

 std::cout<<"Valores Reales  :" <<std::endl;
 std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;
 std::cout<<std::endl;
 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


 std::cout<<std::endl;
 std::cout<<"Valores Ideal: "<<std::endl;
 TIEMPO_GET(HTH_AXPY_OMP) = TIEMPO_GET(HTH_AXPY_SEC) / 2;
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;

 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;

}
// ----------------------------------------------------------------------------
"""
text_file = open("code_axpy.cpp", "w")
text_file.write(code)
text_file.close()

In [61]:
!g++ -o axpy -fopenmp code_axpy.cpp

In [63]:
%env OMP_NUM_THREADS=2
!./axpy 1.0 50000 20

env: OMP_NUM_THREADS=2
 Y : [7, 0, 4, 6, 4, 2, 6, 0, 7, 1, 5, 8, 4, 6, 9, 9, 1, 0, 4, 5, 4, 3, 1, 5, 2, 5, 3, 5, 6, 6, 3, 5, 6, 9, 4, 3, 1, 0, 5, 0, 3, 1, 9, 7, 7, 0, 6, 1, 0, 3, 6, 6, 6, 7, 2, 8, 5, 7, 5, 3, 3, 1, 9, 1, 0, 5, 6, 4, 5, 2, 4, 8, 3, 3, 5, 2, 4, 4, 3, 6, 7, 1, 3, 5, 1, 7, 3, 6, 4, 1, 1, 9, 4, 2, 2, 6, 7, 9, 0, 4, 1, 5, 4, 6, 8, 0, 0, 4, 6, 6, 1, 5, 7, 4, 0, 8, 1, 5, 6, 7, 6, 0, 8, 2, 2, 0, 9, 2, 1, 9, 6, 2, 4, 3, 0, 5, 3, 3, 9, 1, 9, 2, 6, 8, 8, 6, 9, 1, 3, 5, 8, 2, 7, 8, 4, 2, 9, 3, 4, 2, 5, 2, 5, 9, 5, 7, 6, 0, 0, 8, 3, 1, 0, 9, 0, 1, 7, 9, 2, 1, 6, 3, 3, 4, 1, 9, 6, 2, 3, 2, 5, 0, 4, 2, 9, 2, 1, 8, 4, 4, 6, 8, 5, 8, 7, 5, 9, 7, 6, 2, 8, 3, 5, 3, 9, 8, 2, 7, 1, 7, 9, 8, 7, 5, 2, 9, 7, 3, 7, 2, 7, 5, 0, 3, 3, 9, 0, 3, 6, 9, 5, 6, 4, 2, 1, 3, 0, 4, 0, 3, 3, 9, 3, 3, 6, 5, 4, 4, 9, 1, 8, 8, 8, 0, 1, 1, 9, 4, 4, 8, 3, 1, 6, 7, 5, 8, 2, 8, 4, 2, 3, 7, 3, 7, 2, 9, 2, 6, 5, 3, 9, 5, 2, 7, 5, 5, 9, 7, 9, 5, 7, 4, 7, 3, 1, 4, 1, 3, 2, 7, 7, 6, 7, 0, 5, 9, 2, 7, 8, 7, 3, 7, 5, 5,